# Supertrend ontwikkeling en berekening voor nieuwe strategie

Bron: https://www.youtube.com/watch?v=JqEjhBD55NY

In [202]:
# Importeer de benodigde bibliotheken
import pandas as pd
from freqtrade.strategy.interface import IStrategy
from pandas import DataFrame
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
from scipy.spatial.distance import cosine
import numpy as np

In [203]:
# bij weergave van een dataframe is de standaard weergave enkele regels van het begin df en enkele regels van het einde df. Met deze optie laat panda's echt alle regels in de dataframe zien.
pd.set_option('display.max_rows', None)
# Turn off copy in dataframe warnings
# pd.set_option('df.is_copy' = False)
pd.set_option('mode.chained_assignment', 'warn')
# if you set a value on a copy, warning will show

In [233]:
# Load data
df = pd.read_json("ETH_USDT-1d.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06


## Add inndicator from freqtrade indicators

In [205]:
df['rsi'] = ta.RSI(df, timeperiod=14)
df.tail(5)

,date,open,high,low,close,volume,rsi
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06,41.990031
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06,40.240749
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06,42.321609
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06,48.128876
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06,46.989786


## Supertrend berekening

Source: https://www.tradingfuel.com/supertrend-indicator-formula-and-calculation/

    Basic upperband = (High + Low)/2 + multiplier * ATR
    Basic lowerband = (High + Low)/2 - multiplier * ATR

In [206]:
# # Eerst ATR achterhalen. Hiervoor gebruik ik de atr functie uit talib via freqtrade
# df['atr'] = ta.ATR(df, timeperiod=7)
# df[['date','open','high','low','atr']].tail(15)


Tot zover kloppen de getallen met de waardes uit Tradingview.
Nu de supertrend berekenen:

Zelf de ATR berekenen volgens de methode uit:  https://www.youtube.com/watch?v=JqEjhBD55NY  
en bron: https://www.youtube.com/watch?v=JqEjhBD55NY

Hieronder wordt een functie gemaakt dat de true range uitrekend over de afgelopen dat. Dit is nog geen average true range over meerdere dagen. De berekening wordt in een functie gestopt dat aangeroepen kan worden.

In [207]:
def tr(df):
    df['previous_close']=df['close'].shift(1)
    df['high-low']=df['high']-df['low']
    df['high-pc']=abs(df['high']-df['previous_close'])
    df['low-pc']=abs(df['low']-df['previous_close'])
    # calculate true range
    tr=df[['high-low','high-pc','low-pc']].max(axis=1)
    return tr

df['tr']=tr(df)
df[['date','open','high','low','tr']].tail(15)

,date,open,high,low,tr
984,2021-05-18,3282.25,3565.71,3240.00,325.71
985,2021-05-19,3375.08,3441.73,1888.00,1553.73
986,2021-05-20,2438.92,2990.05,2155.00,835.05
987,2021-05-21,2767.87,2938.54,2101.00,837.54
988,2021-05-22,2430.98,2485.00,2145.00,340.00
989,2021-05-23,2295.45,2381.57,1728.74,652.83
990,2021-05-24,2096.63,2675.00,2079.94,595.06
991,2021-05-25,2647.92,2750.00,2378.54,371.46
992,2021-05-26,2705.10,2910.00,2643.00,267.00
993,2021-05-27,2884.82,2889.74,2633.68,256.06


In [208]:
# Tulip ATR
df['atr'] = ta.ATR(df, timeperiod=14)

def my_atr(df,period=14):
    df['tr']=tr(df)
    my_atr = df['tr'].rolling(period).mean()
    df['my_atr']=my_atr
    # print(df)
    return(my_atr)

my_atr(df)
df[['date','close','atr','my_atr','tr']].tail(7)

,date,close,atr,my_atr,tr
992,2021-05-26,2884.94,484.454366,594.152143,267.00
993,2021-05-27,2742.23,468.140483,574.244286,256.06
994,2021-05-28,2412.10,465.778305,570.897143,435.07
995,2021-05-29,2278.74,458.949141,561.477143,370.17
996,2021-05-30,2385.98,447.302059,544.330714,295.89
997,2021-05-31,2706.15,447.391912,541.723571,448.56
998,2021-06-01,2634.57,430.882490,533.905714,216.26


Mijn eigen ATR klopt totaal niet met de TULIP atr. Die gebruik ik daarom verder niet meer.

In [214]:
def supertrend(df, period=7,multiplier=3):
    # Create values for supertrend calculation
    hl2 = ((df['high'] + df['low'])/2)
    df['atr'] = ta.ATR(df, timeperiod=period)
    df['atr_upperband'] = hl2 + (multiplier*df['atr'])
    df['atr_lowerband'] = hl2 - (multiplier*df['atr'])
    df['upperband'] = hl2 + (multiplier*df['atr'])
    df['lowerband'] = hl2 - (multiplier*df['atr'])
    df['in_uptrend'] = True
    
    # Create the supertrend indicator
    for current in range(1, len(df.index)):
        previous = current - 1
        # check if the price is in an uptrend or downtrend
        if df['close'][current] > df['upperband'][previous]:
            # we are in an uptrend
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            # we are in a downtrend
            df['in_uptrend'][current] = False
        else:
            # no change, so we stay in the current trend
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            # if in an uptrend, move the lowerband up to the most recent higher lowerband value
            # (moves lowerband higher in uptrend)
            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]
            # if not in an uptrend, move the upperband to the most recent lower upperband value
            # (moves upperband lower in a downtrend)
            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

supertrend(df)
# print(df[['date','open','high','low','close','atr','tr','upperband','lowerband','in_uptrend']].tail(50))
df[['date','open','high','low','close','atr','tr','upperband','lowerband','in_uptrend']].tail(150)
# df[['date','open','high','low','close','atr','tr','upperband','lowerband','in_uptrend']]


/opt/freqtrade/.env/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/freqtrade/.env/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/freqtrade/.env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/freqtrade/.env/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWith

,date,open,high,low,close,atr,tr,upperband,lowerband,in_uptrend
849,2021-01-03,774.44,1011.07,768.71,978.28,77.844919,242.36,1123.424758,656.355242,True
850,2021-01-04,978.33,1162.97,890.00,1041.43,105.719931,272.97,1343.644792,709.325208,True
851,2021-01-05,1041.45,1134.60,974.45,1099.56,113.495655,160.15,1395.011965,714.038035,True
852,2021-01-06,1100.16,1213.00,1057.26,1208.42,119.530561,155.74,1493.721684,776.538316,True
853,2021-01-07,1208.75,1289.00,1131.00,1224.87,125.026195,158.00,1585.078586,834.921414,True
854,2021-01-08,1224.86,1273.75,1065.00,1216.93,136.986739,208.75,1580.335217,834.921414,True
855,2021-01-09,1216.72,1304.37,1171.36,1276.00,136.418633,133.01,1647.120900,834.921414,True
856,2021-01-10,1276.00,1348.33,1170.00,1254.25,142.405971,178.33,1686.382914,834.921414,True
857,2021-01-11,1254.24,1260.00,915.00,1087.01,171.347976,345.00,1601.543927,834.921414,True
858,2021-01-12,1087.00,1150.02,1006.33,1050.16,167.396836,143.69,1580.365509,834.921414,True


The code above is inspired by the following pinescript:

```
//Author - Rajandran R
//www.marketcalls.in
study("Supertrend V1.0 - Buy or Sell Signal", overlay = true)

Factor=input(3, minval=1,maxval = 100)
Pd=input(7, minval=1,maxval = 100)


Up=hl2-(Factor*atr(Pd))
Dn=hl2+(Factor*atr(Pd))


TrendUp=close[1]>TrendUp[1]? max(Up,TrendUp[1]) : Up
TrendDown=close[1]<TrendDown[1]? min(Dn,TrendDown[1]) : Dn

Trend = close > TrendDown[1] ? 1: close< TrendUp[1]? -1: nz(Trend[1],1)
Tsl = Trend==1? TrendUp: TrendDown

linecolor = Trend == 1 ? green : red

plot(Tsl, color = linecolor , style = line , linewidth = 2,title = "SuperTrend")

plotshape(cross(close,Tsl) and close>Tsl , "Up Arrow", shape.triangleup,location.belowbar,green,0,0)
plotshape(cross(Tsl,close) and close<Tsl , "Down Arrow", shape.triangledown , location.abovebar, red,0,0)
//plot(Trend==1 and Trend[1]==-1,color = linecolor, style = circles, linewidth = 3,title="Trend")

plotarrow(Trend == 1 and Trend[1] == -1 ? Trend : na, title="Up Entry Arrow", colorup=lime, maxheight=60, minheight=50, transp=0)
plotarrow(Trend == -1 and Trend[1] == 1 ? Trend : na, title="Down Entry Arrow", colordown=red, maxheight=60, minheight=50, transp=0)
plot(Up)
plot(Dn)
```

**Supertrend werkt!!**

Alleen die rare foutmelding zit me dwars...

In [210]:
df.index

RangeIndex(start=0, stop=999, step=1)

In [211]:
for current in range(1, len(df.index)):
    previous = current - 1
    
    print(current,df['close'][current],previous,df['close'][previous])

2 90.38 91 93.05
93 93.26 92 90.38
94 89.91 93 93.26
95 87.76 94 89.91
96 89.41 95 87.76
97 86.34 96 89.41
98 83.82 97 86.34
99 83.76 98 83.82
100 84.67 99 83.76
101 94.02 100 84.67
102 100.39 101 94.02
103 98.74 102 100.39
104 114.05 103 98.74
105 107.69 104 114.05
106 115.2 105 107.69
107 129.03 106 115.2
108 138.03 107 129.03
109 127.6 108 138.03
110 129.35 109 127.6
111 113.8 110 129.35
112 135.3 111 113.8
113 132.89 112 135.3
114 137.77 113 132.89
115 131.45 114 137.77
116 139.1 115 131.45
117 152.01 116 139.1
118 146.3 117 152.01
119 151.97 118 146.3
120 152.83 119 151.97
121 153.86 120 152.83
122 149.34 121 153.86
123 147.6 122 149.34
124 148.1 123 147.6
125 124.95 124 148.1
126 124.65 125 124.95
127 123.35 126 124.65
128 114.16 127 123.35
129 126.95 128 114.16
130 118.95 129 126.95
131 121.48 130 118.95
132 121.54 131 121.48
133 119.17 132 121.54
134 122.6 133 119.17
135 117.47 134 122.6
136 115.63 135 117.47
137 117.81 136 115.63
138 116.44 137 117.81
139 116.39 138 116.44
140

Van het bovenstaande supertrendscript probeer ik nu mijn eigen versie te maken.

In [212]:
df['prev_close'] = df['close'].shift(1)
df[['close','prev_close']].tail(5)

,close,prev_close
994,2412.10,2742.23
995,2278.74,2412.10
996,2385.98,2278.74
997,2706.15,2385.98
998,2634.57,2706.15


### Poging 1

In [228]:
def my_supertrend(df, period=7,multiplier=3):
    # Create values for supertrend calculation
    hl2 = ((df['high'] + df['low'])/2)
    df['atr'] = ta.ATR(df, timeperiod=period)
    df['atr_upperband'] = hl2 + (multiplier*df['atr'])
    df['atr_lowerband'] = hl2 - (multiplier*df['atr'])
    df['upperband'] = hl2 + (multiplier*df['atr'])
    df['lowerband'] = hl2 - (multiplier*df['atr'])
    df['in_uptrend'] = True
    
    # Create the supertrend indicator
    # Define some new columns
    df['prev_close'] = df['close'].shift(1)
    df['prev_upperband'] = df['upperband'].shift(1)
    df['prev_lowerband'] = df['lowerband'].shift(1)
    df['in_uptrend'] = False
    df['prev_in_uptrend'] = False
    df['in_downtrend'] = False
    df['prev_in_downtrend'] = False
    df['drawtrend'] = df['upperband']


    # def intrend(df):
    if df['close'].iloc[0] > df['prev_upperband'].iloc[0]: df['in_uptrend'].iloc[0] = True
    elif df['close'].iloc[0] < df['prev_lowerband'].iloc[0]: df['in_uptrend'].iloc[0] = False
    else: df['in_uptrend'].iloc[0] = df['prev_in_uptrend'].iloc[0]
    # return df

    if df['in_uptrend'].iloc[0] and df['lowerband'].iloc[0] < df['prev_lowerband'].iloc[0]: df['drawtrend'].iloc[0] = df['prev_lowerband'].iloc[0]
    if not df['in_uptrend'].iloc[0] and df['upperband'].iloc[0] < df['prev_upperband'].iloc[0]: df['drawtrend'].iloc[0] = df['prev_upperband'].iloc[0]
    # intrend(df)

    
    # if df['close'].iloc[0] > df['prev_upperband'].iloc[0]:
    #     df['in_uptrend'] = True
    # elif df['close'].iloc[0] < df['prev_lowerband'].iloc[0]:
    #     df['in_uptrend'] = False
    # else:
    #     df['in_uptrend'].iloc[0] = df['prev_in_uptrend']

        # if df['in_uptrend'] and df['lowerband'] < df['prev_lowerband']:
        #     df['drawtrend'] = df['prev_lowerband']
        # if not df['in_uptrend'] and df['upperband'] > df['prev_upperband']:
        #     df['drawtrend'] = df['prev_lowerband']
    
    # Create the supertrend indicator
    # for current in range(1, len(df.index)):
    #     previous = current - 1
    #     # check if the price is in an uptrend or downtrend
    #     if df['close'][current] > df['upperband'][previous]:
    #         # we are in an uptrend
    #         df['in_uptrend'][current] = True
        # elif df['close'][current] < df['lowerband'][previous]:
        #     # we are in a downtrend
        #     df['in_uptrend'][current] = False
        # else:
        #     # no change, so we stay in the current trend
        #     df['in_uptrend'][current] = df['in_uptrend'][previous]

            # # if in an uptrend, move the lowerband up to the most recent higher lowerband value
            # # (moves lowerband higher in uptrend)
            # if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
            #     df['lowerband'][current] = df['lowerband'][previous]
            # # if not in an uptrend, move the upperband to the most recent lower upperband value
            # # (moves upperband lower in a downtrend)
            # if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
            #     df['upperband'][current] = df['upperband'][previous]
        
                    
    return df

my_supertrend(df)

# df[['date','close','prev_close','upperband','prev_upperband','lowerband','prev_lowerband','in_uptrend','prev_in_uptrend','in_downtrend','prev_in_downtrend','drawtrend']].tail(150)
df[['date','open','close','prev_close','upperband','prev_upperband','lowerband','prev_lowerband','in_uptrend','drawtrend']].tail(150)
# df[['date','close','in_uptrend']].tail(150)

/opt/freqtrade/.env/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,date,open,close,prev_close,upperband,prev_upperband,lowerband,prev_lowerband,in_uptrend,drawtrend
850,2021-01-04,978.33,1041.43,978.28,1343.644792,1123.424758,709.325208,656.355242,False,1343.644792
851,2021-01-05,1041.45,1099.56,1041.43,1395.011965,1343.644792,714.038035,709.325208,False,1395.011965
852,2021-01-06,1100.16,1208.42,1099.56,1493.721684,1395.011965,776.538316,714.038035,False,1493.721684
853,2021-01-07,1208.75,1224.87,1208.42,1585.078586,1493.721684,834.921414,776.538316,False,1585.078586
854,2021-01-08,1224.86,1216.93,1224.87,1580.335217,1585.078586,758.414783,834.921414,False,1580.335217
855,2021-01-09,1216.72,1276.00,1216.93,1647.120900,1580.335217,828.609100,758.414783,False,1647.120900
856,2021-01-10,1276.00,1254.25,1276.00,1686.382914,1647.120900,831.947086,828.609100,False,1686.382914
857,2021-01-11,1254.24,1087.01,1254.25,1601.543927,1686.382914,573.456073,831.947086,False,1601.543927
858,2021-01-12,1087.00,1050.16,1087.01,1580.365509,1601.543927,575.984491,573.456073,False,1580.365509
859,2021-01-13,1050.36,1129.89,1050.16,1557.372579,1580.365509,567.757421,575.984491,False,1557.372579


### Poging 2